In [133]:
from accounts.models import User
from articles.models import Article, Category, FFM, Tag
from django.contrib.admin.models import LogEntry
from django.contrib.auth.models import Group, Permission
from django.contrib.contenttypes.models import ContentType
from django.contrib.sessions.models import Session
from silk.models import Profile, Request, Response, SQLQuery
from django.db.models.expressions import RawSQL
# Shell Plus Django Imports
from django.core.cache import cache
from django.conf import settings
from django.contrib.auth import get_user_model
from django.db import transaction
from django.db.models import Avg, Case, Count, F, Max, Min, Prefetch, Q, Sum, When, Exists, OuterRef, Subquery
from django.utils import timezone
from django.urls import reverse
from django.db import models


In [13]:
from django.db.models.functions import Coalesce
from django.db.models import Sum, Value

# 検索

In [64]:
target = FFM.objects.get(name='G')
target

DEBUG (0.001) SELECT "articles_ffm"."id", "articles_ffm"."depth", "articles_ffm"."queue", "articles_ffm"."name" FROM "articles_ffm" WHERE "articles_ffm"."name" = 'G'; args=('G',)
DEBUG (0.001) SELECT "articles_ffm"."id", "articles_ffm"."depth", "articles_ffm"."queue", "articles_ffm"."name" FROM "articles_ffm" WHERE "articles_ffm"."name" = 'G'; args=('G',)


<FFM: (2, 10): G>

### 先祖ノードの検索

In [69]:
sub1 = FFM.objects.filter(queue__lt=target.queue,depth__lt=target.depth).values('depth').annotate(max_queue=Max('queue'))

In [75]:
target = FFM.objects.get(name='G')

DEBUG (0.001) SELECT "articles_ffm"."id", "articles_ffm"."depth", "articles_ffm"."queue", "articles_ffm"."name" FROM "articles_ffm" WHERE "articles_ffm"."name" = 'G'; args=('G',)
DEBUG (0.001) SELECT "articles_ffm"."id", "articles_ffm"."depth", "articles_ffm"."queue", "articles_ffm"."name" FROM "articles_ffm" WHERE "articles_ffm"."name" = 'G'; args=('G',)


In [76]:
FFM.objects.filter(queue__in=Subquery(sub1.values('max_queue')))

DEBUG (0.000) SELECT "articles_ffm"."id", "articles_ffm"."depth", "articles_ffm"."queue", "articles_ffm"."name" FROM "articles_ffm" WHERE "articles_ffm"."queue" IN (SELECT MAX(U0."queue") AS "max_queue" FROM "articles_ffm" U0 WHERE (U0."depth" < 2 AND U0."queue" < 10) GROUP BY U0."depth")  LIMIT 21; args=(2, 10)
DEBUG (0.000) SELECT "articles_ffm"."id", "articles_ffm"."depth", "articles_ffm"."queue", "articles_ffm"."name" FROM "articles_ffm" WHERE "articles_ffm"."queue" IN (SELECT MAX(U0."queue") AS "max_queue" FROM "articles_ffm" U0 WHERE (U0."depth" < 2 AND U0."queue" < 10) GROUP BY U0."depth")  LIMIT 21; args=(2, 10)


<FFMQuerySet [<FFM: (0, 0): 0>, <FFM: (1, 2): 0>]>

###  親ノード検索

In [62]:
sub1 = FFM.objects.filter(queue__lt=target.queue,depth=target.depth-1).values('depth').annotate(max_queue=Max('queue'))

In [63]:
FFM.objects.filter(queue__in=Subquery(sub1.values('max_queue')))

DEBUG (0.001) SELECT "articles_ffm"."id", "articles_ffm"."depth", "articles_ffm"."queue", "articles_ffm"."name" FROM "articles_ffm" WHERE "articles_ffm"."queue" IN (SELECT MAX(U0."queue") AS "max_queue" FROM "articles_ffm" U0 WHERE (U0."depth" = 0 AND U0."queue" < 2) GROUP BY U0."depth")  LIMIT 21; args=(0, 2)
DEBUG (0.001) SELECT "articles_ffm"."id", "articles_ffm"."depth", "articles_ffm"."queue", "articles_ffm"."name" FROM "articles_ffm" WHERE "articles_ffm"."queue" IN (SELECT MAX(U0."queue") AS "max_queue" FROM "articles_ffm" U0 WHERE (U0."depth" = 0 AND U0."queue" < 2) GROUP BY U0."depth")  LIMIT 21; args=(0, 2)


<FFMQuerySet [<FFM: (0, 0): A>]>

### 先祖ノード検索の一般化

n 代前まで取得するような query

In [51]:
n=2
sub1 = FFM.objects.filter(queue__lt=target.queue,depth__lt=target.depth, depth__gte=target.depth-n).values('depth')\
    .annotate(max_queue=Max('queue'))
FFM.objects.filter(queue__in=Subquery(sub1.values('max_queue')))

DEBUG (0.000) SELECT "articles_ffm"."id", "articles_ffm"."depth", "articles_ffm"."queue", "articles_ffm"."name" FROM "articles_ffm" WHERE "articles_ffm"."queue" IN (SELECT MAX(U0."queue") AS "max_queue" FROM "articles_ffm" U0 WHERE (U0."depth" >= 0 AND U0."depth" < 2 AND U0."queue" < 9) GROUP BY U0."depth")  LIMIT 21; args=(0, 2, 9)
DEBUG (0.000) SELECT "articles_ffm"."id", "articles_ffm"."depth", "articles_ffm"."queue", "articles_ffm"."name" FROM "articles_ffm" WHERE "articles_ffm"."queue" IN (SELECT MAX(U0."queue") AS "max_queue" FROM "articles_ffm" U0 WHERE (U0."depth" >= 0 AND U0."depth" < 2 AND U0."queue" < 9) GROUP BY U0."depth")  LIMIT 21; args=(0, 2, 9)


<FFMQuerySet [<FFM: (0, 0): A>, <FFM: (1, 2): C>]>

## 子孫ノードの検索

In [52]:
target = FFM.objects.get(name='B')
target

DEBUG (0.000) SELECT "articles_ffm"."id", "articles_ffm"."depth", "articles_ffm"."queue", "articles_ffm"."name" FROM "articles_ffm" WHERE "articles_ffm"."name" = 'B'; args=('B',)
DEBUG (0.000) SELECT "articles_ffm"."id", "articles_ffm"."depth", "articles_ffm"."queue", "articles_ffm"."name" FROM "articles_ffm" WHERE "articles_ffm"."name" = 'B'; args=('B',)


<FFM: (1, 1): B>

In [53]:
# x はダミー列
sub1 = FFM.objects.annotate(x=RawSQL("'0'",[])).values('x').\
    filter(queue__gt=target.queue, depth__lte=target.depth).\
    annotate(min_queue=Min('queue'))
sub1

DEBUG (0.000) SELECT ('0') AS "x", MIN("articles_ffm"."queue") AS "min_queue" FROM "articles_ffm" WHERE ("articles_ffm"."depth" <= 1 AND "articles_ffm"."queue" > 1) GROUP BY ('0')  LIMIT 21; args=(1, 1)
DEBUG (0.000) SELECT ('0') AS "x", MIN("articles_ffm"."queue") AS "min_queue" FROM "articles_ffm" WHERE ("articles_ffm"."depth" <= 1 AND "articles_ffm"."queue" > 1) GROUP BY ('0')  LIMIT 21; args=(1, 1)


<FFMQuerySet [{'x': '0', 'min_queue': 2}]>

In [54]:
FFM.objects.filter(depth__gt=target.depth,
                               queue__gt=target.queue, 
                               queue__lt=Subquery(sub1.values('min_queue')[:1]))

DEBUG (0.000) SELECT "articles_ffm"."id", "articles_ffm"."depth", "articles_ffm"."queue", "articles_ffm"."name" FROM "articles_ffm" WHERE ("articles_ffm"."depth" > 1 AND "articles_ffm"."queue" > 1 AND "articles_ffm"."queue" < (SELECT MIN(U0."queue") AS "min_queue" FROM "articles_ffm" U0 WHERE (U0."depth" <= 1 AND U0."queue" > 1) GROUP BY ('0')  LIMIT 1))  LIMIT 21; args=(1, 1, 1, 1)
DEBUG (0.000) SELECT "articles_ffm"."id", "articles_ffm"."depth", "articles_ffm"."queue", "articles_ffm"."name" FROM "articles_ffm" WHERE ("articles_ffm"."depth" > 1 AND "articles_ffm"."queue" > 1 AND "articles_ffm"."queue" < (SELECT MIN(U0."queue") AS "min_queue" FROM "articles_ffm" U0 WHERE (U0."depth" <= 1 AND U0."queue" > 1) GROUP BY ('0')  LIMIT 1))  LIMIT 21; args=(1, 1, 1, 1)


<FFMQuerySet []>

### n代先までの子孫ノードの検索一般化

In [55]:
INT_MAX = 0x7fffffff

In [56]:
n = 1
target = FFM.objects.get(name='C')

DEBUG (0.000) SELECT "articles_ffm"."id", "articles_ffm"."depth", "articles_ffm"."queue", "articles_ffm"."name" FROM "articles_ffm" WHERE "articles_ffm"."name" = 'C'; args=('C',)
DEBUG (0.000) SELECT "articles_ffm"."id", "articles_ffm"."depth", "articles_ffm"."queue", "articles_ffm"."name" FROM "articles_ffm" WHERE "articles_ffm"."name" = 'C'; args=('C',)


In [57]:
# x はダミー列
sub1 = FFM.objects.annotate(x=RawSQL("'0'",[])).values('x').\
    filter(queue__gt=target.queue, depth__lte=target.depth).\
    annotate(min_queue=Min('queue'))

FFM.objects.filter(depth__gt=target.depth,
                               depth__lte=target.depth+n,
                               queue__gt=target.queue, 
                               queue__lt=Coalesce(Subquery(sub1.values('min_queue')[:1]),Value(INT_MAX)))


DEBUG (0.001) SELECT "articles_ffm"."id", "articles_ffm"."depth", "articles_ffm"."queue", "articles_ffm"."name" FROM "articles_ffm" WHERE ("articles_ffm"."depth" > 1 AND "articles_ffm"."depth" <= 2 AND "articles_ffm"."queue" > 2 AND "articles_ffm"."queue" < (COALESCE((SELECT MIN(U0."queue") AS "min_queue" FROM "articles_ffm" U0 WHERE (U0."depth" <= 1 AND U0."queue" > 2) GROUP BY ('0')  LIMIT 1), 2147483647)))  LIMIT 21; args=(1, 2, 2, 1, 2, 2147483647)
DEBUG (0.001) SELECT "articles_ffm"."id", "articles_ffm"."depth", "articles_ffm"."queue", "articles_ffm"."name" FROM "articles_ffm" WHERE ("articles_ffm"."depth" > 1 AND "articles_ffm"."depth" <= 2 AND "articles_ffm"."queue" > 2 AND "articles_ffm"."queue" < (COALESCE((SELECT MIN(U0."queue") AS "min_queue" FROM "articles_ffm" U0 WHERE (U0."depth" <= 1 AND U0."queue" > 2) GROUP BY ('0')  LIMIT 1), 2147483647)))  LIMIT 21; args=(1, 2, 2, 1, 2, 2147483647)


<FFMQuerySet [<FFM: (2, 3): D>, <FFM: (2, 4): E>, <FFM: (2, 8): F>, <FFM: (2, 9): G>]>

## 考察

先祖や子孫ノードの検索が容易. これで更新が 1クエリ? 2クエリ で行えるのは強い.

# ノードの追加

あるノード(A)の子供を追加するとき,  Aより大きい `queue` のノードに `queue+1` の操作を行う.  
これにより `A.queue+1` の queue があくので, そこに新しいノードを登録する.

In [59]:
from django.db.models import Exists,Count

In [60]:
parent = FFM.objects.get(name='E')

DEBUG (0.001) SELECT "articles_ffm"."id", "articles_ffm"."depth", "articles_ffm"."queue", "articles_ffm"."name" FROM "articles_ffm" WHERE "articles_ffm"."name" = 'E'; args=('E',)
DEBUG (0.001) SELECT "articles_ffm"."id", "articles_ffm"."depth", "articles_ffm"."queue", "articles_ffm"."name" FROM "articles_ffm" WHERE "articles_ffm"."name" = 'E'; args=('E',)


In [61]:
with transaction.atomic():  # start transaction
    FFM.objects.filter(queue__gt=parent.queue).update(queue=F('queue')+1)
    params = {
        'queue' : parent.queue+1,
        'depth' : parent.depth+1,
        'name' : f"{parent.name}'s child"
    }
    FFM.objects.create(**params)

DEBUG (0.000) BEGIN; args=None
DEBUG (0.000) BEGIN; args=None
DEBUG (0.001) UPDATE "articles_ffm" SET "queue" = ("articles_ffm"."queue" + 1) WHERE "articles_ffm"."queue" > 4; args=(1, 4)
DEBUG (0.001) UPDATE "articles_ffm" SET "queue" = ("articles_ffm"."queue" + 1) WHERE "articles_ffm"."queue" > 4; args=(1, 4)
DEBUG (0.000) INSERT INTO "articles_ffm" ("depth", "queue", "name") VALUES (3, 5, 'E''s child'); args=[3, 5, "E's child"]
DEBUG (0.000) INSERT INTO "articles_ffm" ("depth", "queue", "name") VALUES (3, 5, 'E''s child'); args=[3, 5, "E's child"]


### 考察

ノードの追加のたびに毎回 update が必要となる.
これを回避するために 以下を考えた.

1. 定数：interval を用意
2. `select MAX(queue)-1 from FFM where queue > parent.queue` を取得. `mq` とする.
3. `parent.queue = mq` のとき, 空きがないので, `parent.queue` より大きい queue をもつノードに `queue+= interval` を行う.
4. 追加ノードの queue を次のように決める
    + 3\. で空きがあったとき:  `mq`
    + 空きがなかったとき, または 2\. が取得できなかったとき: `mq+interavl`
5. 追加ノードの登録

# ノードの削除

ノードの削除は3パターンある.

1. 指定ノードを部分木ごと削除する
2. 指定ノードの子孫ノードだけ削除する
3. 指定ノードを削除して、その子孫ノードを繰り上げる

1, 2 は (指定ノードと)子孫ノードの検索を行いすべて削除するだけなので, 1クエリで可能.

3 は 

+ \(1) 指定ノードを削除
+ \(2) 子孫ノードの depth を -1

という 2 つのクエリが必要.

以下, 3 のみ試す.

In [134]:
target = FFM.objects.get(name='G')

DEBUG (0.001) SELECT "articles_ffm"."id", "articles_ffm"."depth", "articles_ffm"."queue", "articles_ffm"."name" FROM "articles_ffm" WHERE "articles_ffm"."name" = 'G'; args=('G',)
DEBUG (0.001) SELECT "articles_ffm"."id", "articles_ffm"."depth", "articles_ffm"."queue", "articles_ffm"."name" FROM "articles_ffm" WHERE "articles_ffm"."name" = 'G'; args=('G',)


In [135]:
with transaction.atomic():  # start transaction
    target.delete()
    
    sub1 = FFM.objects.annotate(x=RawSQL("'0'",[])).values('x').\
        filter(queue__gt=target.queue, depth__lte=target.depth).\
        annotate(min_queue=Min('queue'))
    update_targets = FFM.objects.filter(depth__gt=target.depth,
                                   queue__gt=target.queue, 
                                   queue__lt=Coalesce(Subquery(sub1.values('min_queue')[:1]),Value(INT_MAX)) 
                                   )
    update_targets.update(depth=F('depth')-Value(1))


DEBUG (0.000) BEGIN; args=None
DEBUG (0.000) BEGIN; args=None
DEBUG (0.003) DELETE FROM "articles_ffm" WHERE "articles_ffm"."id" IN (7); args=(7,)
DEBUG (0.003) DELETE FROM "articles_ffm" WHERE "articles_ffm"."id" IN (7); args=(7,)
DEBUG (0.000) UPDATE "articles_ffm" SET "depth" = ("articles_ffm"."depth" - 1) WHERE ("articles_ffm"."depth" > 2 AND "articles_ffm"."queue" > 6 AND "articles_ffm"."queue" < (COALESCE((SELECT MIN(U0."queue") AS "min_queue" FROM "articles_ffm" U0 WHERE (U0."depth" <= 2 AND U0."queue" > 6) GROUP BY ('0')  LIMIT 1), 2147483647))); args=(1, 2, 6, 2, 6, 2147483647)
DEBUG (0.000) UPDATE "articles_ffm" SET "depth" = ("articles_ffm"."depth" - 1) WHERE ("articles_ffm"."depth" > 2 AND "articles_ffm"."queue" > 6 AND "articles_ffm"."queue" < (COALESCE((SELECT MIN(U0."queue") AS "min_queue" FROM "articles_ffm" U0 WHERE (U0."depth" <= 2 AND U0."queue" > 6) GROUP BY ('0')  LIMIT 1), 2147483647))); args=(1, 2, 6, 2, 6, 2147483647)


## おまけ

ツリー表示

In [136]:
nodes = FFM.objects.all()

tbl_info = nodes.aggregate(d_min=Min('depth'), d_max=Max('depth'), q_min=Min('queue'), q_max=Max('queue'))

row_size = tbl_info['d_max'] - tbl_info['d_min'] +1
col_size = tbl_info['q_max'] - tbl_info['q_min'] +1

tbl = [[''] * col_size for _ in range(row_size)]

for node in nodes:
    i = node.depth - tbl_info['d_min']
    j = node.queue - tbl_info['q_min']
    tbl[i][j] = node.name
    if i > 0:
        tbl[i-1][j] = '+'

DEBUG (0.000) SELECT MIN("articles_ffm"."depth") AS "d_min", MAX("articles_ffm"."depth") AS "d_max", MIN("articles_ffm"."queue") AS "q_min", MAX("articles_ffm"."queue") AS "q_max" FROM "articles_ffm"; args=()
DEBUG (0.000) SELECT MIN("articles_ffm"."depth") AS "d_min", MAX("articles_ffm"."depth") AS "d_max", MIN("articles_ffm"."queue") AS "q_min", MAX("articles_ffm"."queue") AS "q_max" FROM "articles_ffm"; args=()
DEBUG (0.001) SELECT "articles_ffm"."id", "articles_ffm"."depth", "articles_ffm"."queue", "articles_ffm"."name" FROM "articles_ffm"; args=()
DEBUG (0.001) SELECT "articles_ffm"."id", "articles_ffm"."depth", "articles_ffm"."queue", "articles_ffm"."name" FROM "articles_ffm"; args=()


In [137]:
for i in range(row_size):
    flg = 0
    for j in range(col_size-1,-1,-1):
        if not tbl[i][j]:
            if flg:
                tbl[i][j] = '-'
        elif tbl[i][j] == '+':
            flg=True
        else:
            flg=False
tbl

[['A', '+', '-', '-', '+', '', '', '', '', '', ''],
 ['', 'B', '+', '+', 'C', '+', '-', '+', '-', '-', '+'],
 ['', '', 'D', 'E', '', 'F', '', 'H', '', '', 'I']]

In [138]:
for r in tbl:
    for j in range(col_size):
        if not r[j]:
            r[j] = ' '
    print(*r)

A + - - +            
  B + + C + - + - - +
    D E   F   H     I
